In [1]:
import os 
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.url import URL
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import boto3
from scipy.stats import pearsonr
from scipy.stats import gmean
import statsmodels.api as sm

In [2]:
import sys
sys.path.append("/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb")
import utils_schema as us
from utils_schema import get_otu_and_taxo

In [3]:
us

<module 'utils_schema' from '/Users/sebastianbedoyamazo/Google Drive/Mi unidad/siwa/iluma-kb/utils_schema.py'>

In [4]:
load_dotenv("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/.env", override=True)

True

In [5]:
os.environ
sys.path.append("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others")
import utils_dataexp as u

Reading aws keys from .env successfully!!


In [6]:
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")

In [7]:
url = URL.create(
    drivername='redshift+redshift_connector',
    host='iluma-kb-1.cn4ff1ztoyt9.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    username='awsuser',
    password='Ilumasiwa1'
)

engine = create_engine(url)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

redshift_metadata = sa.MetaData(bind=session.bind)

# metadata for merge with perfomance (projectid, house, pen)

In [8]:
query = "SELECT * FROM microbiome as M LEFT JOIN kit as K on M.kitid = K.kitid LEFT JOIN animal as A on M.animalid = A.animalid;"
metadata = pd.read_sql(query, engine)
print(metadata.shape)
print(metadata.columns)
metadata.head(3)

(3799, 19)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved', 'kitid', 'projectid',
       'age', 'treatment', 'treatmentnumber', 'client', 'animalid',
       'animaltype', 'animalnumber', 'house', 'pen'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,H001_01F,H001_01F-M,H001,20181130,H001_01,feces,2.429103,27.0,H001,E267,14,Negative control,1,Iluma,H001_01,Broiler,01,1.0,4.0
1,H001_01I,H001_01I-M,H001,20181130,H001_01,ileum,1.339185,47.0,H001,E267,14,Negative control,1,Iluma,H001_01,Broiler,01,1.0,4.0
2,H001_04C,H001_04C-M,H001,20181130,H001_04,cecum,6.082133,125.0,H001,E267,14,Negative control,1,Iluma,H001_04,Broiler,04,1.0,29.0


# Project select

In [9]:
df_metadata = metadata.loc[metadata['projectid'].isin(['E267', 'E271', 'E345', 'E347'])]
print(df_metadata.columns)
print(df_metadata.shape)
df_metadata.head()

Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved', 'kitid', 'projectid',
       'age', 'treatment', 'treatmentnumber', 'client', 'animalid',
       'animaltype', 'animalnumber', 'house', 'pen'],
      dtype='object')
(1038, 19)


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,H001_01F,H001_01F-M,H001,20181130,H001_01,feces,2.429103,27.0,H001,E267,14,Negative control,1,Iluma,H001_01,Broiler,01,1.0,4.0
1,H001_01I,H001_01I-M,H001,20181130,H001_01,ileum,1.339185,47.0,H001,E267,14,Negative control,1,Iluma,H001_01,Broiler,01,1.0,4.0
2,H001_04C,H001_04C-M,H001,20181130,H001_04,cecum,6.082133,125.0,H001,E267,14,Negative control,1,Iluma,H001_04,Broiler,04,1.0,29.0
3,H001_04F,H001_04F-M,H001,20181130,H001_04,feces,2.705841,20.0,H001,E267,14,Negative control,1,Iluma,H001_04,Broiler,04,1.0,29.0
4,H001_05F,H001_05F-M,H001,20181130,H001_05,feces,0.940546,9.0,H001,E267,14,Negative control,1,Iluma,H001_05,Broiler,05,3.0,24.0


In [10]:
df_metadata['pen'] = df_metadata['pen'].astype(float).astype(int)
df_metadata['house'] = df_metadata['house'].astype(float).astype(int)

/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_1530/3885707156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata['pen'] = df_metadata['pen'].astype(float).astype(int)
/var/folders/8_/97xnhkcj3qvf45tl2cgtc7vc0000gp/T/ipykernel_1530/3885707156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata['house'] = df_metadata['house'].astype(float).astype(int)


# metadata merge with perfomance

In [11]:
df_pilot = pd.read_excel("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/df_pilot.xlsx")
df_pilot.head(1)

,Projectid,House,Pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42,FCR0_42
0,E267,1,1,5.0,2857.307692,719.230769,2811.993407,1.618182,1.652339


In [14]:
df_pilot.rename(columns = {'House':'house', 'Pen':'pen', 'Projectid':'projectid'}, inplace = True)
#df_pilot['house'] = df_pilot['house'].replace('[^0-9\.]', '', regex=True)
df_pilot['house'] = df_pilot['house'].astype(float).astype(int)
df_pilot['pen'] = df_pilot['pen'].astype(float).astype(int)

In [15]:
df_metadata_per = df_metadata.merge(df_pilot, how='left', left_on =['projectid', 'house', 'pen'], right_on =['projectid', 'house', 'pen'])
print(df_metadata_per.shape)
print(df_metadata_per.columns)
df_metadata_per.head(1)

(1038, 25)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved', 'kitid', 'projectid',
       'age', 'treatment', 'treatmentnumber', 'client', 'animalid',
       'animaltype', 'animalnumber', 'house', 'pen', 'Trt', 'BW42', 'BWG36_42',
       'BWG0_42', 'FCR36_42', 'FCR0_42'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,...,animaltype,animalnumber,house,pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42,FCR0_42
0,H001_01F,H001_01F-M,H001,20181130,H001_01,feces,2.429103,27.0,H001,E267,...,Broiler,01,1,4,1.0,2985.0,740.0,2940.515152,1.632883,1.58598


In [16]:
metadata_test = df_metadata_per[['sampleid', 'fullsampleid', 'animalid',
       'samplelocation', 'projectid', 'Trt', 'BW42',
       'FCR0_42', 'FCR36_42', 'BWG0_42', 'BWG36_42']]
metadata_test

,sampleid,fullsampleid,animalid,animalid,samplelocation,projectid,Trt,BW42,FCR0_42,FCR36_42,BWG0_42,BWG36_42
0,H001_01F,H001_01F-M,H001_01,H001_01,feces,E267,1.0,2985.000000,1.58598,1.632883,2940.515152,740.000000
1,H001_01I,H001_01I-M,H001_01,H001_01,ileum,E267,1.0,2985.000000,1.58598,1.632883,2940.515152,740.000000
2,H001_04C,H001_04C-M,H001_04,H001_04,cecum,E267,1.0,3015.600000,1.614089,1.919226,2972.314286,703.200000
3,H001_04F,H001_04F-M,H001_04,H001_04,feces,E267,1.0,3015.600000,1.614089,1.919226,2972.314286,703.200000
4,H001_05F,H001_05F-M,H001_05,H001_05,feces,E267,1.0,2989.200000,1.574259,1.746961,2948.542857,724.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1033,0123_06I,0123_06I-M,0123_06,0123_06,ileum,E347,9.0,2536.000000,NaN,1.968026,2491.76,625.500000
1034,0124_01I,0124_01I-M,0124_01,0124_01,ileum,E347,10.0,2506.500000,NaN,1.950207,2459,602.500000
1035,0124_02I,0124_02I-M,0124_02,0124_02,ileum,E347,10.0,2545.000000,NaN,1.853818,2504.8,596.666667
1036,0124_04I,0124_04I-M,0124_04,0124_04,ileum,E347,10.0,2480.500000,NaN,1.937128,2433.58,588.500000


In [17]:
metadata_test.to_excel('metadata_test.xlsx', index=False)

# otu_table and taxonomy for age 42

In [18]:
samples = list(df_metadata['sampleid'])
len(samples)

1038

In [19]:
otu_table, taxonomy = us.get_otus_taxo_fromsamples(samples)

Shape otu table (6492, 1038)
Shape taxonomy (6492, 8)


In [20]:
otu_table.head(3)

,0105_01C,0105_01F,0105_01I,0105_02C,0105_02F,0105_02I,0105_03C,0105_03F,0105_03I,0105_04C,...,H029_03I,H029_04I,H029_05C,H029_05I,H029_06I,H029_07C,H029_07F,H029_08C,H029_08F,H029_09C
0008a288f7c98ac22b6f496a93ae01e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0011ac4d2a454f4451c263073131dfb3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00176c6f0de3e7f6822cf8fa80411ac5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
otu_table_test = otu_table.reset_index(drop=False)
otu_table_test.rename(columns = {'index':'OTU'}, inplace = True)

In [22]:
otu_table_test

,OTU,0105_01C,0105_01F,0105_01I,0105_02C,0105_02F,0105_02I,0105_03C,0105_03F,0105_03I,...,H029_03I,H029_04I,H029_05C,H029_05I,H029_06I,H029_07C,H029_07F,H029_08C,H029_08F,H029_09C
0,0008a288f7c98ac22b6f496a93ae01e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0011ac4d2a454f4451c263073131dfb3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00176c6f0de3e7f6822cf8fa80411ac5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0017bbccd79e6bdc565512bbe82894c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00197a84bc2b537f6df9075b3ec3d3cb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,946.0,0.0,0.0,0.0,0.0,76.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6487,ffc0f243dc37b9c2cd6f05e4f519f2d9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6488,ffcd1beef68b56e896ffcf260acb80d2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6489,fff30ab0f76e35d83e7c42cf88c9b853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6490,fff66b75302d7abe37de4c2d97031f48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
otu_table_test.to_excel('otu_table_test.xlsx', index=False)

In [25]:
taxonomy.rename(columns = {'otu':'OTU'}, inplace = True)
taxonomy.head()

,OTU,species,genus,family,order,tclass,phylum,kingdom
0,8e248f6dda97209ac7fb30767c3c4a20,UNKNOWN,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria
1,54c340319d4dc2fea14ac168f9b80c6e,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Bacilli,Firmicutes,Bacteria
2,ee4e01883525e1c99d4c87f2cd7b66f6,UNKNOWN,Butyricicoccus,Butyricicoccaceae,Oscillospirales,Clostridia,Firmicutes,Bacteria
3,526b61ab9c895df9800a505056344f67,Anaerostipes_butyraticus,Anaerostipes,Lachnospiraceae,Lachnospirales,Clostridia,Firmicutes,Bacteria
4,57f8b56747cbdaa624a96cd2d6872344,UNKNOWN,Lactobacillus,Lactobacillaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria


In [26]:
taxonomy.to_excel('taxonomy_test.xlsx', index=False)